In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,CuDNNLSTM
print(tf.__version__)

1.14.0


In [2]:
xnormal = []
ynormal = []


# Reading HeartRate data for Normal Patients
#f = 
#f = 
#f = 
f = [open("./NormalData/OxygenSaturation.csv", "r"), open("./NormalData/HeartRate.csv", "r"), open("./NormalData/BloodPressure.csv", "r"), open("./NormalData/RespiratoryRate.csv", "r")]


s1=f[0].readline()
s2=f[1].readline()
s3=f[2].readline()
s4=f[3].readline()


#print(s1)
#print(s2)
#print(s3)
#print(s4)
while s1!="":
    a = []
    b=s1.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
        #print(b[i])
        #print("a="+str(a))
    b=s2.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    b=s3.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    b=s4.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    xnormal.append(a)
    ynormal.append(0)
    #print(len(a))
    s1=f[0].readline()
    s2=f[1].readline()
    s3=f[2].readline()
    s4=f[3].readline()
    

print("Normal: "+str(len(xnormal))+"  "+str(len(xnormal[0]))+" Y dimension:"+str(len(ynormal)))    

f = [open("./AbnormalData/OxygenSaturation.csv", "r"), open("./AbnormalData/HeartRate.csv", "r"), open("./AbnormalData/BloodPressure.csv", "r"), open("./AbnormalData/RespiratoryRate.csv", "r")]

xAbnormal=[]
yAbnormal=[]

s1=f[0].readline()
s2=f[1].readline()
s3=f[2].readline()
s4=f[3].readline()


#print(s1)
#print(s2)
#print(s3)
#print(s4)
while s1!="":
    a = []
    b=s1.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
        #print(b[i])
        #print("a="+str(a))
    b=s2.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    b=s3.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    b=s4.split(',')
    for i in range(0, len(b)):
        a.append(float(b[i]))
    xAbnormal.append(a)
    yAbnormal.append(1)
    #print(len(a))
    s1=f[0].readline()
    s2=f[1].readline()
    s3=f[2].readline()
    s4=f[3].readline()
    

print("Abnormal: "+str(len(xAbnormal))+"  "+str(len(xAbnormal[0]))+" Y dimension:"+str(len(yAbnormal)))

Normal: 1558  1024 Y dimension:1558
Abnormal: 461  1024 Y dimension:461


In [3]:
cTrainx=[] # current train x data
cTrainy=[] # current test y data

for i in range(0, 461):
    cTrainx.append(xnormal[i])
    cTrainy.append(ynormal[i])
print(str(len(cTrainx))+"  "+str(len(cTrainx[0]))+" Y dimension:"+str(len(cTrainy)))
i=0
for i in range(0, 460):
    cTrainx.append(xAbnormal[i])
    cTrainy.append(yAbnormal[i])
print(str(len(cTrainx))+"  "+str(len(cTrainx[0]))+" Y dimension:"+str(len(cTrainy)))

461  1024 Y dimension:461
921  1024 Y dimension:921


In [4]:
# List to numpy array conversion
y=np.transpose(cTrainy)
x = np.array(cTrainx)
print(x.shape)
x = x.reshape(len(x),len(x[0]),1);
print(x.shape)
#print(x)
print(y.shape)

(921, 1024)
(921, 1024, 1)
(921,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)
x_train = tf.keras.utils.normalize(X_train, axis=0)
x_test = tf.keras.utils.normalize(X_test, axis=0)
print(x_train.shape)
#print(x_train)
#y_train=np.array([1,1])
print(y_train.shape)
#print(y_train)
count=0
for op in y_train:
    if(op==1):
        count+=1
print("Total normal data after distribution: "+str(len(y_train)-count)+"\nTotal Abnormal data: "+str(count))

(736, 1024, 1)
(736,)
Total normal data after distribution: 381
Total Abnormal data: 355


In [10]:
print(x_train.shape)
print(x_train[0].shape)

model = Sequential()
print("Input shape")
print(x_train.shape[1:])

#model.add(LSTM(1024, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))

model.add(CuDNNLSTM(256, input_shape=(x_train.shape[1:]), return_sequences=True))

#model.add(CuDNNLSTM(256))
#model.add(CuDNNLSTM(256, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dense(256, activation='sigmoid'))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.01, decay=1e-5)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)



(737, 1024, 1)
(1024, 1)
Input shape
(1024, 1)


In [11]:
for j in range(0, 3):
    cTrainx=[] # current train x data
    cTrainy=[] # current test y data

    for i in range(0, 461):
        cTrainx.append(xnormal[i+j*461])
        cTrainy.append(ynormal[i+j*461])
    #print(str(len(cTrainx))+"  "+str(len(cTrainx[0]))+" Y dimension:"+str(len(cTrainy)))
    i=0
    for i in range(0, 461):
        cTrainx.append(xAbnormal[i])
        cTrainy.append(yAbnormal[i])
    #print(str(len(cTrainx))+"  "+str(len(cTrainx[0]))+" Y dimension:"+str(len(cTrainy)))

    # List to numpy array conversion
    y=np.transpose(cTrainy)
    x = np.array(cTrainx)
    x = x.reshape(len(x),len(x[0]),1);
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=50)
    x_train = tf.keras.utils.normalize(X_train, axis=0)
    x_test = tf.keras.utils.normalize(X_test, axis=0)
    count=0
    for op in y_train:
        if(op==1):
            count+=1

    model.fit(x_train,
              y_train,
              epochs=3,
              validation_data=(x_test, y_test))

Train on 737 samples, validate on 185 samples
Epoch 1/3
737/737 [==============================] - 3s 4ms/sample - loss: 1.2593 - acc: 0.5020 - val_loss: 0.8040 - val_acc: 0.4324
Epoch 2/3
737/737 [==============================] - 3s 4ms/sample - loss: 0.8384 - acc: 0.4830 - val_loss: 0.7196 - val_acc: 0.5676
Epoch 3/3
737/737 [==============================] - 3s 4ms/sample - loss: 0.7413 - acc: 0.5088 - val_loss: 0.8247 - val_acc: 0.4324
Train on 737 samples, validate on 185 samples
Epoch 1/3
737/737 [==============================] - 3s 4ms/sample - loss: 0.7400 - acc: 0.4898 - val_loss: 0.7114 - val_acc: 0.4324
Epoch 2/3
737/737 [==============================] - 3s 4ms/sample - loss: 0.7034 - acc: 0.4925 - val_loss: 1.3121 - val_acc: 0.4324
Epoch 3/3
737/737 [==============================] - 3s 4ms/sample - loss: 1.0028 - acc: 0.5061 - val_loss: 0.7374 - val_acc: 0.4324
Train on 737 samples, validate on 185 samples
Epoch 1/3
737/737 [==============================] - 3s 4ms/samp